### To determine the regions of interest in the analysis dataset

In [ ]:
#Import required modules
import pybedtools, os
import pandas as pd
import numpy as np

In [ ]:
def sort_chrom(Chr):
    """ Sort BED file in alpha-numeric order by chromosome.

    Arguments
    ---------
        Chr: Chromosome (i.e. chr1)

    Returns
    -------
        New: New variable to order autosomal chromosomes as well as sex and mitochondrial chromosomes.
    """

    if Chr:
        New = Chr[3:]
        if New == 'X':
            New = 23
        elif New == 'Y':
            New = 24
        elif New == 'M':
            New = 25
        elif New == 'N':
            New = 26
        else:
            New = int(New)
    else:
        New = 0
    return New

In [ ]:
# There is no CNV, but at least TWO experimental designs have MORE than ONE probes matching exon region --> TRUE NEGATIVE
def regions_of_interestv(exonfile, gtexome, arrayid1, arrayid2, arrayid3, out):
    #Check that the output path exists, if not create it
    if not os.path.exists(out):
        os.makedirs(out)
        
    # Create a BedTool object from the exon file 
    exon = pybedtools.BedTool(exonfile)
    # Create a BedTool object from the ground truth CNVs matching coding exons
    gt = pybedtools.BedTool(gtexome)
    
    #Identify which exons are the exons that are the CNVs of ground truth
    gt_exon_isec = exon.intersect(gt, wao=True).intersect(gt, c=True)
    gt_exon_ = [(f[0],f[1],f[2],f[3],f[4],f[5],f[9],f[21],f[29]) for f in gt_exon_isec]
    gt_exon = pybedtools.BedTool(gt_exon_)                   
    
    ## Calculate the number of probes from each experimental design matching exon regions
    #estd20
    exon_probestudy1A = gt_exon.intersect(arrayid1[0], c=True)
    exon_probestudy1B = gt_exon.intersect(arrayid1[1], c=True)
    exon_probestudy1C = gt_exon.intersect(arrayid1[2], c=True)
    exon_probestudy1D = gt_exon.intersect(arrayid1[3], c=True)
    cat_probesid1 = exon_probestudy1A.cat(exon_probestudy1B, postmerge=False).cat(
        exon_probestudy1C, postmerge=False).cat(exon_probestudy1D, postmerge=False).sort()
    exon_probestudy1 = cat_probesid1.groupby(g=[1,2,3,4,5,6,7,8,9], c=10, o=["sum"])
    #nstd46
    exon_probestudy2 = gt_exon.intersect(arrayid2, c=True)
    #estd195
    exon_probestudy3 = gt_exon.intersect(arrayid3, c=True)
    
    # Keep only the field containing the number of probes
    num_probe_study1_exon = [x[9] for x in exon_probestudy1]
    num_probe_study2_exon = [x[9] for x in exon_probestudy2]
    num_probe_study3_exon = [x[9] for x in exon_probestudy3]
    
    # Create a dataframe that stores all information obtained for each exon interval
    data = pybedtools.BedTool.to_dataframe(gt_exon, names=['#chrom', 'start', 'end', 'gene', 
                                                           'transcript','exon', 'CNV_type', 'confidence_score',
                                                           'match_CNV']) 

    data.insert(loc = 9, column = 'num_probes_id1', value = num_probe_study1_exon)
    data.insert(loc = 10, column = 'num_probes_id2', value = num_probe_study2_exon)
    data.insert(loc = 11, column = 'num_probes_id3', value = num_probe_study3_exon)
         
    N = []
    P = []
    no_capture = []
    ROI = []
    for index, row in data.iterrows():
        chrom = row['#chrom']
        start = int(row['start'])
        end = int(row['end'])
        gene_name = row['gene']
        gene_id = row['transcript']
        gene_num = row['exon']
        cnv_type = row['CNV_type']
        conf_score = row['confidence_score']
        match_cnv = int(row['match_CNV']) #0 -> not match; num -> match
        probes_id1 = int(row['num_probes_id1'])
        probes_id2 = int(row['num_probes_id2'])
        probes_id3 = int(row['num_probes_id3'])
        
        #if match_cnv == 0 (negative), check at least two experimental designs have probes matching exon region
        #If there are at least two experimental designs, it implies that the CNV could have been characterized; 
        #therefore, it is negative. If there is insufficient probe, or there is no congruence between studies, 
        #it is not considered negative. 
        #Two methods are examined: restrictive and highly restrictive. The highly restrictive is the one analyzed 
        #(more than 1 probe in at least 2 experimental designs). The restrictive one is not considered in the final work 
        #(at least 1 probe in at least 1 experimental design).
        if match_cnv == 0:
            roi = 'negative'
            ###RESTRICTIVE:
            #if (probes_id1 != 0 and probes_id2 != 0) or (probes_id1 != 0 and probes_id3 != 0) or (probes_id2 != 0 and probes_id3 != 0):
            ###HIGH RESTRICTIVE:
            if (probes_id1>1 and probes_id2>1) or (probes_id1>1 and probes_id3>1) or (probes_id2>1 and probes_id3>1):
                N.append((chrom, start, end, gene_name, gene_id, gene_num, cnv_type, conf_score, match_cnv, 
                          probes_id1, probes_id2, probes_id3, roi))
                ROI.append((chrom, start, end, gene_name, gene_id, gene_num, cnv_type, conf_score, match_cnv, 
                          probes_id1, probes_id2, probes_id3, roi))
            else:
                #pass
                no_capture.append((chrom, start, end, gene_name, gene_id, gene_num, cnv_type, conf_score, match_cnv, 
                          probes_id1, probes_id2, probes_id3, roi))
        else:
            roi = 'positive'
            P.append((chrom, start, end, gene_name, gene_id, gene_num, cnv_type, conf_score, match_cnv, 
                          probes_id1, probes_id2, probes_id3, roi))
            ROI.append((chrom, start, end, gene_name, gene_id, gene_num, cnv_type, conf_score, match_cnv, 
                          probes_id1, probes_id2, probes_id3, roi))
            
    N_sorted = sorted(N, key=lambda x: (x[0], x[1], x[2]))
    P_sorted = sorted(P, key=lambda x: (x[0], x[1], x[2]))
    no_capture_sort = sorted(no_capture, key=lambda x: (x[0], x[1], x[2]))
    ROI_sorted = sorted(ROI, key=lambda x: (x[0], x[1], x[2]))
    
    # Write the BED file
    with open(os.path.join(out, "negative.bed"), 'w') as n:
        n.write('#chrom\tstart\tend\tgene\ttranscript\texon\tcnv_type\tconf_score\tmatch_cnv\t\
        probes_id1\tprobes_id2\tprobes_id3\tclass\n')
        n.write("\n".join(map(lambda x: "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % 
                              (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12]), 
                              N_sorted)))
    with open(os.path.join(out, "negative_no_capture.bed"), 'w') as nn:
        nn.write('#chrom\tstart\tend\tgene\ttranscript\texon\tcnv_type\tconf_score\tmatch_cnv\t\
        probes_id1\tprobes_id2\tprobes_id3\tclass\n')
        nn.write("\n".join(map(lambda x: "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % 
                              (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12]), 
                      no_capture_sort)))
    with open(os.path.join(out, "positive.bed"), 'w') as p:
        p.write('#chrom\tstart\tend\tgene\ttranscript\texon\tcnv_type\tconf_score\tmatch_cnv\t\
        probes_id1\tprobes_id2\tprobes_id3\tclass\n')
        p.write("\n".join(map(lambda x: "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % 
                              (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12]), 
                              P_sorted)))
    with open(os.path.join(out, "ROI.bed"), 'w') as roi:
        roi.write('#chrom\tstart\tend\tgene\ttranscript\texon\tcnv_type\tconf_score\tmatch_cnv\t\
        probes_id1\tprobes_id2\tprobes_id3\tclass\n')
        roi.write("\n".join(map(lambda x: "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % 
                              (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12]), 
                              ROI_sorted)))


In [ ]:
regions_of_interest('exome_INGEMM.no_pseudo.bed', 'ground_truth_exome_indet.bed', 
                    ['nimblegen_10M_A.bed.gz', 'nimblegen_10M_B.bed.gz', 
                     'nimblegen_10M_C.bed.gz', 'nimblegen_10M_D.bed.gz'],
                    ['A-GEOD-11386_hg19.bed', 'A-GEOD-11387_hg19.bed'],
                    ['Human1Mv1_C-b37.strand.bed', 'GenomeWideSNP_6.hg19_parsed.bed'], 
                    'roi_highrestrict')